# Pytorch Note 01 (CNN)

### OUTLINE

1. Dataset
2. DataLoader
3. Declaration of Model
4. Initialization Before Training
5. Training
6. Important Comments

In [12]:
import torch
import numpy as np

In [13]:
LR = 0.001
BATCH_SIZE = 256
EPOCH = 40

In [8]:
def loadData(filename, mode):
    x = []
    y = []
    f = open(filename, 'r')
    row = csv.reader(f, delimiter=' ')
    n_row = 0
    for r in row:
        if n_row != 0:
            temp = []
            for i in range(len(r)):
                if i == 0:
                    if mode == 'train': 
                        y.append(int(r[0][0]))
                        temp.append(int(r[0][2:]))
                    else:
                        temp.append(int(r[0][len(str(n_row)) + 1:]))
                else:
                    temp.append(int(r[i]))
            x.append(temp)
        n_row += 1
    f.close()
    
    x = np.array(x)
    y = np.array(y)
    x = x.reshape(x.shape[0], 1, 48, 48)
    x = x / 255

    if mode == 'train': return x, y
    else: return x

### Dataset

In [9]:
from torch.utils.data import Dataset

class Data(Dataset):
    def __init__(self, filename, train, transform=None):
        self.x, self.y = loadData(filename, train)
        self.x = self.x.astype(np.float32)                               #[1]
        self.transform = transform

    def __getitem__(self, index):
        x = self.x[index]
        if self.transform is not None:
            x = self.transform(x)
        return x, self.y[index]

    def __len__(self):
        return self.x.shape[0]

\__init__(self, ...): Get data and initialize the attributes of Dataset.


\__getitem__(self, index): Do the transforms and return x, y with index.

\__len__(self): Return the number of datas.

### DataLoader

In [11]:
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor

data = Data('train.csv', 'True', ToTensor())
train_loader = DataLoader(data, batch_size=BATCH_SIZE, shuffle=True)

### Declaration of Model

In [5]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 60, 5)
        self.conv2 = nn.Conv2d(60, 120, 3)
        self.conv3 = nn.Conv2d(120, 240, 3)

        self.l1 = nn.Linear(4*4*240, 400)
        self.out = nn.Linear(400, 7)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv3(x)), (2, 2))
        x = x.view(x.size(0), -1)
        x = F.relu(self.l1(x))
        output = self.out(x)                                              #[2]
        return output

\__init__(self): Declarations of the layers.

forward(self, x): How the variable go through the model, and return the output (the prediction of the data).

### Initialization Before Training

In [6]:
from torch.optim import Adam

model = CNN().float()                                                     #[1]
model.cuda() #Move the model to gpu.

optim = Adam(model.parameters(), lr=LR) #Declaration of optimizer.
loss_function = nn.CrossEntropyLoss() #Declaration of loss_function.      #[2]
loss_function.cuda() #Move the loss_function to gpu.

### Training

In [18]:
from torch.autograd import Variable

for epoch in range(EPOCH):
    for iter, (x, y) in enumerate(train_loader):                          #[3]
        #Only Variable can compute gradients in Pytorch 0.3.0.
        #Move the datas to gpu.
        x = Variable(x.cuda())
        y = Variable(y.cuda()).long()                                     #[4]
        
        optim.zero_grad() #Clear the gradients of the optimizer
        output = model(x) #Get the prediction of the model
        loss = loss_function(output, y) #Get the loss
        loss.backward() #Get the gradients
        optim.step() #Update the weights in the optimizer
        
        if (iter + 1) % 5 == 0:
            print('Epoch:', epoch + 1, '| Iter:', iter + 1, '| train loss:%.4f' %loss.data[0])

### Important Comments

[1] If the datas are DoubleTensor, executing time would increase explosively in "x = Variable(x.cuda())". Therefore the datas were suggested to be transform into float.

[2] torch.nn.CrossEntropyLoss does softmax automatically, so there's no need to do softmax in the last layer.

[3] While getting datas in DataLoader, it would call \__getitem__ in Dataset.

[4] The target of torch.nn.CrossEntropyLoss() must be int64.